In [154]:
import pandas as pd
# Importa libreria geopandas como gpd
import geopandas as gpd

# Extracción de las MANZANAS en Shapefile

In [155]:
# Lee archivo shapefile de distritos del norte de Irlanda.
# Antes hay que asegurarse de crear la carpeta Shapefiles y subir todos los archivos de Shapefiles.zip
gdf_districts = gpd.read_file('MANZANA.shp') # Distritos del norte de Irlanda

# muestra el tipo de dato
type(gdf_districts)

geopandas.geodataframe.GeoDataFrame

In [156]:
gdf_Lima = gdf_districts.copy()
gdf_Lima = gdf_Lima[(gdf_Lima.DEPARTAMEN == 'LIMA') & (gdf_Lima.PROVINCIA == 'LIMA')]

In [157]:
gdf_Lima.head(5)

,IDMANZANA,CODDPTO,CODPROV,CODDIST,CODZONA,SUFZONA,CODMZNA,SUFMZNA,UBIGEO,CODCCPP,DEPARTAMEN,PROVINCIA,DISTRITO,NOMCCCPP,IDCCPP,Shape_STAr,Shape_STLe,geometry
340925,150101013000450,15,01,01,013,00,045,0,150101,0001,LIMA,LIMA,LIMA,LIMA,1501010001,3.140281e-07,0.002454,"POLYGON ((-77.06513 -12.05393, -77.06549 -12.0..."
340926,150101013000410,15,01,01,013,00,041,0,150101,0001,LIMA,LIMA,LIMA,LIMA,1501010001,4.670634e-07,0.003969,"POLYGON ((-77.06667 -12.0535, -77.0663 -12.053..."
340927,150101013000350,15,01,01,013,00,035,0,150101,0001,LIMA,LIMA,LIMA,LIMA,1501010001,6.790691e-08,0.001773,"POLYGON ((-77.06712 -12.05334, -77.06711 -12.0..."
340928,150101013000370,15,01,01,013,00,037,0,150101,0001,LIMA,LIMA,LIMA,LIMA,1501010001,1.303425e-07,0.002416,"POLYGON ((-77.06729 -12.05376, -77.06739 -12.0..."
340929,150101013000340,15,01,01,013,00,034,0,150101,0001,LIMA,LIMA,LIMA,LIMA,1501010001,6.984726e-08,0.001533,"POLYGON ((-77.06751 -12.05394, -77.06758 -12.0..."


In [158]:
gdf_Lima.shape

(107100, 18)

# Extracción de Albergues potenciales

## Centros recreacionales

In [159]:
df_recreacion = pd.read_excel('Equipamiento de recreación pública (PLANMET 2040).xlsx')

In [160]:
print('Registros de Centros recreacionales en BRUTO: ',df_recreacion.shape[0])

Registros de Centros recreacionales en BRUTO:  6655


In [161]:
df_recreacion.head(5)

,OBJECTID,COD_EQ,ID_MANZANA,IDLOTE,IDMZNAR,COD_SECT,TIPO_EQ,SUBTIPO_EQ,NOM_EQ,AREA_M2,OBSERV,NOMBDIST,NOMBPROV,NOMBDEP,Shape__Area,Shape__Length
0,1,RE-1042,150139002000130,NaN,NaN,NaN,RECREACION PUBLICA,PLAZA,Plaza Miami,8521.913596,CEMENTO,SANTA ROSA,LIMA,LIMA,8521.91,420.40
1,2,RE-1043,150139004000190,NaN,NaN,NaN,RECREACION PUBLICA,PLAZA PRINCIPAL,Plaza de Armas de Santa Rosa,3521.627604,CEMENTO,SANTA ROSA,LIMA,LIMA,3521.63,257.03
2,3,RE-1044,150125010000490,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,NaN,2521.858226,VERDE,PUENTE PIEDRA,LIMA,LIMA,2521.86,397.11
3,4,RE-1045,15012504800011D,NaN,NaN,NaN,RECREACION PUBLICA,PLAZA,Plaza Virgen de la Medalla Milagrosa,5417.728509,VERDE,PUENTE PIEDRA,LIMA,LIMA,5417.73,329.34
4,5,RE-1046,150125002000480,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,Plaza Jerusalén,4727.502509,VERDE,PUENTE PIEDRA,LIMA,LIMA,4727.50,275.62


In [162]:
df_recreacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6655 entries, 0 to 6654
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   OBJECTID       6655 non-null   int64  
 1   COD_EQ         6654 non-null   object 
 2   ID_MANZANA     5530 non-null   object 
 3   IDLOTE         0 non-null      float64
 4   IDMZNAR        169 non-null    object 
 5   COD_SECT       0 non-null      float64
 6   TIPO_EQ        6654 non-null   object 
 7   SUBTIPO_EQ     6655 non-null   object 
 8   NOM_EQ         3019 non-null   object 
 9   AREA_M2        6655 non-null   float64
 10  OBSERV         6654 non-null   object 
 11  NOMBDIST       6655 non-null   object 
 12  NOMBPROV       6654 non-null   object 
 13  NOMBDEP        6654 non-null   object 
 14  Shape__Area    6655 non-null   float64
 15  Shape__Length  6655 non-null   float64
dtypes: float64(5), int64(1), object(10)
memory usage: 832.0+ KB


In [163]:
df_recreacion = df_recreacion.rename(columns={
    'Shape__Area': 'Shape_Area',
    'Shape__Length': 'Shape_Length'
})

In [164]:
# Convertir la columna ID_MANZANA a string
df_recreacion['ID_MANZANA'] = df_recreacion['ID_MANZANA'].astype(str)

In [165]:
# CRUZAR DATOS DE SHAPEFILE
# Realizar el inner join entre gdf_Lima y df_recreacion_filter
# La clave de unión es "ID_MANZANA"
gdf_recreacion = df_recreacion.merge(gdf_Lima, left_on='ID_MANZANA', right_on='IDMANZANA', how='inner')

In [166]:
print('Centros recreacionales que tienen Shapefile: ', gdf_recreacion.shape[0])

Centros recreacionales que tienen Shapefile:  5471


In [167]:
gdf_recreacion.sample(5)

,OBJECTID,COD_EQ,ID_MANZANA,IDLOTE,IDMZNAR,COD_SECT,TIPO_EQ,SUBTIPO_EQ,NOM_EQ,AREA_M2,...,UBIGEO,CODCCPP,DEPARTAMEN,PROVINCIA,DISTRITO,NOMCCCPP,IDCCPP,Shape_STAr,Shape_STLe,geometry
601,684,RE-1593,150110063000400,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,Parque N° 2,7472.479477,...,150110,0001,LIMA,LIMA,COMAS,LA LIBERTAD,1501100001,2.124230e-06,0.005996,"POLYGON ((-77.05584 -11.9557, -77.05585 -11.95..."
1126,1266,RE-3352,150132052010040,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,NaN,646.298955,...,150132,0001,LIMA,LIMA,SAN JUAN DE LURIGANCHO,SAN JUAN DE LURIGANCHO,1501320001,5.530029e-08,0.000960,"POLYGON ((-76.97311 -11.966, -76.97311 -11.965..."
3069,3443,RE-4502,15010700700053K,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,NaN,1465.015299,...,150107,0001,LIMA,LIMA,CHACLACAYO,CHACLACAYO,1501070001,2.679461e-07,0.002234,"POLYGON ((-76.77954 -11.98612, -76.7795 -11.98..."
4517,5243,RE-6144,15014001100020C,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,NaN,1715.281122,...,150140,0001,LIMA,LIMA,SANTIAGO DE SURCO,SANTIAGO DE SURCO,1501400001,1.399311e-06,0.006139,"POLYGON ((-76.96822 -12.13006, -76.96879 -12.1..."
511,584,RE-1493,15012503700038D,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,NaN,437.600362,...,150125,0001,LIMA,LIMA,PUENTE PIEDRA,PUENTE PIEDRA,1501250001,4.388556e-08,0.000970,"POLYGON ((-77.0696 -11.88595, -77.06982 -11.88..."


In [168]:
# DATOS QUE NO SE CRUZAN
# Realizar el outer join entre gdf_Lima y df_recreacion_filter
gdf_result = df_recreacion.merge(gdf_Lima, left_on='ID_MANZANA', right_on='IDMANZANA', how='outer', indicator=True)

# Filtrar los registros que no encontraron coincidencia
no_match_recreacion = gdf_result[gdf_result['_merge'] == 'left_only']

In [169]:
no_match_recreacion_rastreable = no_match_recreacion.dropna(subset=['NOM_EQ'])

In [170]:
print('Centros recreacionales que NO tienen Shapefile, pero son rastreables: ', no_match_recreacion_rastreable.shape[0])

Centros recreacionales que NO tienen Shapefile, pero son rastreables:  234


In [171]:
no_match_recreacion_rastreable.head(5)

,OBJECTID,COD_EQ,ID_MANZANA,IDLOTE,IDMZNAR,COD_SECT,TIPO_EQ,SUBTIPO_EQ,NOM_EQ,AREA_M2,...,CODCCPP,DEPARTAMEN,PROVINCIA,DISTRITO,NOMCCCPP,IDCCPP,Shape_STAr,Shape_STLe,geometry,_merge
7,8.0,RE-1049,nan,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,Parque Miraflores,1596.316754,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,left_only
9,42.0,RE-4862,nan,NaN,NaN,NaN,RECREACION PUBLICA,PLAZUELA,Plazuela Inca Garcilaso de la Vega,179.870590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,left_only
10,64.0,RE-4868,nan,NaN,MZ-0000007,NaN,RECREACION PUBLICA,PLAZUELA,Plazuela Jardin de Santa Rosa,1613.930194,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,left_only
11,68.0,RE-4872,nan,NaN,NaN,NaN,RECREACION PUBLICA,PLAZUELA,Plazuela San Agustín,648.665773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,left_only
12,71.0,RE-4875,nan,NaN,NaN,NaN,RECREACION PUBLICA,PLAZUELA,Plazuela Las Limeñitas,966.531211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,left_only


## Centros deportivos

In [172]:
df_deportivo = pd.read_excel('Equipamiento deportivo (PLANMET 2040).xlsx')

In [173]:
df_deportivo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4216 entries, 0 to 4215
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OBJECTID      4216 non-null   int64  
 1   COD_EQ        4216 non-null   object 
 2   ID_MANZANA    3939 non-null   object 
 3   IDLOTE        0 non-null      float64
 4   IDMZNAR       0 non-null      float64
 5   COD_SECT      0 non-null      float64
 6   TIPO_EQ       4216 non-null   object 
 7   SUBTIPO_EQ    4216 non-null   object 
 8   NOM_EQ        370 non-null    object 
 9   AREA_M2       4216 non-null   float64
 10  OBSERV        0 non-null      float64
 11  NOMBDIST      4216 non-null   object 
 12  NOMBPROV      4216 non-null   object 
 13  NOMBDEP       4216 non-null   object 
 14  NOM_UA        0 non-null      float64
 15  Shape_Area    4216 non-null   float64
 16  Shape_Length  4216 non-null   float64
dtypes: float64(8), int64(1), object(8)
memory usage: 560.1+ KB


In [174]:
# Eliminar la columna NOM_UA del DataFrame
df_deportivo = df_deportivo.drop(columns=['NOM_UA'])

In [175]:
# Convertir la columna ID_MANZANA a string
df_deportivo['ID_MANZANA'] = df_deportivo['ID_MANZANA'].astype(str)

In [176]:
# CRUZAR DATOS DE SHAPEFILE
# Realizar el inner join entre gdf_Lima y df_recreacion_filter
# La clave de unión es "ID_MANZANA"
gdf_deportivo = df_deportivo.merge(gdf_Lima, left_on='ID_MANZANA', right_on='IDMANZANA', how='inner')

In [177]:
print('Centros deportivos que tienen Shapefile: ', gdf_deportivo.shape[0])

Centros deportivos que tienen Shapefile:  3939


In [178]:
gdf_deportivo.head(5)

,OBJECTID,COD_EQ,ID_MANZANA,IDLOTE,IDMZNAR,COD_SECT,TIPO_EQ,SUBTIPO_EQ,NOM_EQ,AREA_M2,...,UBIGEO,CODCCPP,DEPARTAMEN,PROVINCIA,DISTRITO,NOMCCCPP,IDCCPP,Shape_STAr,Shape_STLe,geometry
0,3,D-2357,150132092000330,NaN,NaN,NaN,OU:DEPORTE,LOSA MULTIDEPORTIVA,NaN,585.850150,...,150132,0001,LIMA,LIMA,SAN JUAN DE LURIGANCHO,SAN JUAN DE LURIGANCHO,1501320001,4.362677e-07,0.003102,"POLYGON ((-77.01442 -11.99518, -77.01446 -11.9..."
1,7,D-1026,150112012000420,NaN,NaN,NaN,OU:DEPORTE,LOSA MULTIDEPORTIVA,NaN,1072.399392,...,150112,0001,LIMA,LIMA,INDEPENDENCIA,INDEPENDENCIA,1501120001,1.751800e-06,0.005418,"POLYGON ((-77.06282 -11.98442, -77.06121 -11.9..."
2,8,D-1027,150112012000420,NaN,NaN,NaN,OU:DEPORTE,CAMPO DEPORTIVO,EL NARANJAL,8491.443048,...,150112,0001,LIMA,LIMA,INDEPENDENCIA,INDEPENDENCIA,1501120001,1.751800e-06,0.005418,"POLYGON ((-77.06282 -11.98442, -77.06121 -11.9..."
3,9,D-1028,150112012000420,NaN,NaN,NaN,OU:DEPORTE,LOSA MULTIDEPORTIVA,NaN,555.717492,...,150112,0001,LIMA,LIMA,INDEPENDENCIA,INDEPENDENCIA,1501120001,1.751800e-06,0.005418,"POLYGON ((-77.06282 -11.98442, -77.06121 -11.9..."
4,615,D-1104,150112012000420,NaN,NaN,NaN,OU:DEPORTE,LOSA MULTIDEPORTIVA,NaN,475.503729,...,150112,0001,LIMA,LIMA,INDEPENDENCIA,INDEPENDENCIA,1501120001,1.751800e-06,0.005418,"POLYGON ((-77.06282 -11.98442, -77.06121 -11.9..."


In [179]:
# DATOS QUE NO SE CRUZAN
# Realizar el outer join entre gdf_Lima y df_recreacion_filter
gdf_result2 = df_deportivo.merge(gdf_Lima, left_on='ID_MANZANA', right_on='IDMANZANA', how='outer', indicator=True)

# Filtrar los registros que no encontraron coincidencia
no_match_deportivo = gdf_result2[gdf_result2['_merge'] == 'left_only']

In [180]:
no_match_deportivo_rastreable = no_match_deportivo.dropna(subset=['NOM_EQ'])

In [181]:
print('Centros deportivos que NO tienen Shapefile, pero son rastreables: ', no_match_deportivo_rastreable.shape[0])

Centros deportivos que NO tienen Shapefile, pero son rastreables:  12


# Agrupamiento de Albergues potenciales: Recreativos + deportivos

In [182]:
# Concatenar los dos DataFrames a nivel de filas
gdf_combined = pd.concat([gdf_recreacion, gdf_deportivo], axis=0, ignore_index=True)

In [183]:
gdf_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9410 entries, 0 to 9409
Data columns (total 34 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   OBJECTID      9410 non-null   int64   
 1   COD_EQ        9410 non-null   object  
 2   ID_MANZANA    9410 non-null   object  
 3   IDLOTE        0 non-null      float64 
 4   IDMZNAR       0 non-null      object  
 5   COD_SECT      0 non-null      float64 
 6   TIPO_EQ       9410 non-null   object  
 7   SUBTIPO_EQ    9410 non-null   object  
 8   NOM_EQ        3143 non-null   object  
 9   AREA_M2       9410 non-null   float64 
 10  OBSERV        5471 non-null   object  
 11  NOMBDIST      9410 non-null   object  
 12  NOMBPROV      9410 non-null   object  
 13  NOMBDEP       9410 non-null   object  
 14  Shape_Area    9410 non-null   float64 
 15  Shape_Length  9410 non-null   float64 
 16  IDMANZANA     9410 non-null   object  
 17  CODDPTO       9410 non-null   object  
 18  CODPROV 

In [184]:
gdf_combined

,OBJECTID,COD_EQ,ID_MANZANA,IDLOTE,IDMZNAR,COD_SECT,TIPO_EQ,SUBTIPO_EQ,NOM_EQ,AREA_M2,...,UBIGEO,CODCCPP,DEPARTAMEN,PROVINCIA,DISTRITO,NOMCCCPP,IDCCPP,Shape_STAr,Shape_STLe,geometry
0,1,RE-1042,150139002000130,NaN,NaN,NaN,RECREACION PUBLICA,PLAZA,Plaza Miami,8521.913596,...,150139,0001,LIMA,LIMA,SANTA ROSA,SANTA ROSA,1501390001,6.997925e-07,0.003825,"POLYGON ((-77.17481 -11.79968, -77.17481 -11.7..."
1,2,RE-1043,150139004000190,NaN,NaN,NaN,RECREACION PUBLICA,PLAZA PRINCIPAL,Plaza de Armas de Santa Rosa,3521.627604,...,150139,0001,LIMA,LIMA,SANTA ROSA,SANTA ROSA,1501390001,4.455260e-07,0.003180,"POLYGON ((-77.15797 -11.78584, -77.15771 -11.7..."
2,3,RE-1044,150125010000490,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,NaN,2521.858226,...,150125,0001,LIMA,LIMA,PUENTE PIEDRA,PUENTE PIEDRA,1501250001,3.431174e-07,0.003262,"POLYGON ((-77.11043 -11.83321, -77.11045 -11.8..."
3,4,RE-1045,15012504800011D,NaN,NaN,NaN,RECREACION PUBLICA,PLAZA,Plaza Virgen de la Medalla Milagrosa,5417.728509,...,150125,0001,LIMA,LIMA,PUENTE PIEDRA,PUENTE PIEDRA,1501250001,5.113287e-07,0.002939,"POLYGON ((-77.08583 -11.92793, -77.08598 -11.9..."
4,5,RE-1046,150125002000480,NaN,NaN,NaN,RECREACION PUBLICA,PARQUE LOCAL,Plaza Jerusalén,4727.502509,...,150125,0001,LIMA,LIMA,PUENTE PIEDRA,PUENTE PIEDRA,1501250001,4.593910e-07,0.002747,"POLYGON ((-77.11757 -11.82812, -77.11778 -11.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9405,4211,D-4150,150128022000420,NaN,NaN,NaN,OU:DEPORTE,COMPLEJO DEPORTIVO,RIMAC,771.507460,...,150128,0001,LIMA,LIMA,RIMAC,RIMAC,1501280001,5.585861e-07,0.002990,"POLYGON ((-77.03137 -12.04144, -77.0306 -12.04..."
9406,4212,D-1021,15012301201039B,NaN,NaN,NaN,OU:DEPORTE,CAMPO DEPORTIVO,VISTA ALEGRE,2117.492086,...,150123,0001,LIMA,LIMA,PACHACAMAC,PACHACAMAC,1501230001,1.683302e-07,0.001679,"POLYGON ((-76.88069 -12.09506, -76.88074 -12.0..."
9407,4213,D-1022,150133029000240,NaN,NaN,NaN,OU:DEPORTE,COMPLEJO DEPORTIVO,IPD SAN JUAN DE MIRAFLORES,54665.688321,...,150133,0001,LIMA,LIMA,SAN JUAN DE MIRAFLORES,CIUDAD DE DIOS,1501330001,1.176255e-05,0.014409,"POLYGON ((-76.97324 -12.1596, -76.9732 -12.158..."
9408,4214,D-1023,150143007000090,NaN,NaN,NaN,OU:DEPORTE,CAMPO DEPORTIVO,NaN,9301.308144,...,150143,0001,LIMA,LIMA,VILLA MARIA DEL TRIUNFO,VILLA MARIA DEL TRIUNFO,1501430001,5.304972e-06,0.010955,"POLYGON ((-76.94306 -12.13771, -76.94314 -12.1..."


In [185]:
# Eliminar filas con valores duplicados en la columna ID_MANZANA
gdf_combined = gdf_combined.drop_duplicates(subset='ID_MANZANA', keep='first')

In [186]:
print("Se tienen ",gdf_combined.shape[0]," albergues potenciales entre centros recreativos y deportivos")

Se tienen  7420  albergues potenciales entre centros recreativos y deportivos


In [ ]:
# Descargar el archivo de los albergues potenciales como SHAPEFILE
gdf_combined = gpd.GeoDataFrame(gdf_combined, geometry='geometry')

In [189]:
# Renombrar columnas largas a nombres más cortos (máximo 10 caracteres)
gdf_combined = gdf_combined.rename(columns={
    'Shape_Length': 'Shape_Len',
    # Agrega más renombramientos si es necesario
})

# Exportar a shapefile
gdf_combined.to_file('albergues_potenciales_lima.shp', driver='ESRI Shapefile')

### ELiminación de columnas sin detalle

In [229]:
# Eliminar múltiples columnas que no guardan información relevante
gdf_combined = gdf_combined.drop(columns=['OBJECTID', 'COD_EQ', 'IDLOTE', 'IDMZNAR', 'COD_SECT', 'AREA_M2', 'IDMANZANA', 'DEPARTAMEN', 'PROVINCIA', 'DISTRITO', 'NOMCCCPP', 'IDCCPP'])

In [230]:
gdf_combined.columns

Index(['ID_MANZANA', 'TIPO_EQ', 'SUBTIPO_EQ', 'NOM_EQ', 'OBSERV', 'NOMBDIST',
       'NOMBPROV', 'NOMBDEP', 'Shape_Area', 'Shape_Len', 'CODDPTO', 'CODPROV',
       'CODDIST', 'CODZONA', 'SUFZONA', 'CODMZNA', 'SUFMZNA', 'UBIGEO',
       'CODCCPP', 'Shape_STAr', 'Shape_STLe', 'geometry'],
      dtype='object')

In [231]:
gdf_combined.head()

,ID_MANZANA,TIPO_EQ,SUBTIPO_EQ,NOM_EQ,OBSERV,NOMBDIST,NOMBPROV,NOMBDEP,Shape_Area,Shape_Len,...,CODDIST,CODZONA,SUFZONA,CODMZNA,SUFMZNA,UBIGEO,CODCCPP,Shape_STAr,Shape_STLe,geometry
0,150139002000130,RECREACION PUBLICA,PLAZA,Plaza Miami,CEMENTO,SANTA ROSA,LIMA,LIMA,8521.91,420.40,...,39,002,00,013,0,150139,0001,6.997925e-07,0.003825,"POLYGON ((-77.17481 -11.79968, -77.17481 -11.7..."
1,150139004000190,RECREACION PUBLICA,PLAZA PRINCIPAL,Plaza de Armas de Santa Rosa,CEMENTO,SANTA ROSA,LIMA,LIMA,3521.63,257.03,...,39,004,00,019,0,150139,0001,4.455260e-07,0.003180,"POLYGON ((-77.15797 -11.78584, -77.15771 -11.7..."
2,150125010000490,RECREACION PUBLICA,PARQUE LOCAL,NaN,VERDE,PUENTE PIEDRA,LIMA,LIMA,2521.86,397.11,...,25,010,00,049,0,150125,0001,3.431174e-07,0.003262,"POLYGON ((-77.11043 -11.83321, -77.11045 -11.8..."
3,15012504800011D,RECREACION PUBLICA,PLAZA,Plaza Virgen de la Medalla Milagrosa,VERDE,PUENTE PIEDRA,LIMA,LIMA,5417.73,329.34,...,25,048,00,011,D,150125,0001,5.113287e-07,0.002939,"POLYGON ((-77.08583 -11.92793, -77.08598 -11.9..."
4,150125002000480,RECREACION PUBLICA,PARQUE LOCAL,Plaza Jerusalén,VERDE,PUENTE PIEDRA,LIMA,LIMA,4727.50,275.62,...,25,002,00,048,0,150125,0001,4.593910e-07,0.002747,"POLYGON ((-77.11757 -11.82812, -77.11778 -11.8..."


### Agregación de Longitud y Latitud

In [234]:
gdf_combined_coor = gdf_combined.copy()

# Calcular los centroides si las geometrías no son puntos
gdf_combined_coor['centroid'] = gdf_combined_coor.geometry.centroid

# Extraer latitud y longitud desde los centroides
gdf_combined_coor['latitud'] = gdf_combined_coor['centroid'].y
gdf_combined_coor['longitud'] = gdf_combined_coor['centroid'].x


C:\Users\amyan\AppData\Local\Temp\ipykernel_27356\1442919720.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_combined_coor['centroid'] = gdf_combined_coor.geometry.centroid


In [235]:
gdf_combined_coor

,ID_MANZANA,TIPO_EQ,SUBTIPO_EQ,NOM_EQ,OBSERV,NOMBDIST,NOMBPROV,NOMBDEP,Shape_Area,Shape_Len,...,CODMZNA,SUFMZNA,UBIGEO,CODCCPP,Shape_STAr,Shape_STLe,geometry,centroid,latitud,longitud
0,150139002000130,RECREACION PUBLICA,PLAZA,Plaza Miami,CEMENTO,SANTA ROSA,LIMA,LIMA,8521.91,420.40,...,013,0,150139,0001,6.997925e-07,0.003825,"POLYGON ((-77.17481 -11.79968, -77.17481 -11.7...",POINT (-77.17448 -11.80046),-11.800459,-77.174476
1,150139004000190,RECREACION PUBLICA,PLAZA PRINCIPAL,Plaza de Armas de Santa Rosa,CEMENTO,SANTA ROSA,LIMA,LIMA,3521.63,257.03,...,019,0,150139,0001,4.455260e-07,0.003180,"POLYGON ((-77.15797 -11.78584, -77.15771 -11.7...",POINT (-77.1574 -11.78613),-11.786128,-77.157402
2,150125010000490,RECREACION PUBLICA,PARQUE LOCAL,NaN,VERDE,PUENTE PIEDRA,LIMA,LIMA,2521.86,397.11,...,049,0,150125,0001,3.431174e-07,0.003262,"POLYGON ((-77.11043 -11.83321, -77.11045 -11.8...",POINT (-77.111 -11.8331),-11.833098,-77.111000
3,15012504800011D,RECREACION PUBLICA,PLAZA,Plaza Virgen de la Medalla Milagrosa,VERDE,PUENTE PIEDRA,LIMA,LIMA,5417.73,329.34,...,011,D,150125,0001,5.113287e-07,0.002939,"POLYGON ((-77.08583 -11.92793, -77.08598 -11.9...",POINT (-77.08597 -11.92755),-11.927554,-77.085969
4,150125002000480,RECREACION PUBLICA,PARQUE LOCAL,Plaza Jerusalén,VERDE,PUENTE PIEDRA,LIMA,LIMA,4727.50,275.62,...,048,0,150125,0001,4.593910e-07,0.002747,"POLYGON ((-77.11757 -11.82812, -77.11778 -11.8...",POINT (-77.11805 -11.82822),-11.828218,-77.118052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9404,150101029000100,OU:DEPORTE,COMPLEJO DEPORTIVO,AMAZONAS,NaN,LIMA,LIMA,LIMA,2637.82,255.74,...,010,0,150101,0001,1.548559e-06,0.008109,"POLYGON ((-77.02187 -12.04415, -77.02089 -12.0...",POINT (-77.02133 -12.04443),-12.044432,-77.021326
9405,150128022000420,OU:DEPORTE,COMPLEJO DEPORTIVO,RIMAC,NaN,RIMAC,LIMA,LIMA,771.51,156.58,...,042,0,150128,0001,5.585861e-07,0.002990,"POLYGON ((-77.03137 -12.04144, -77.0306 -12.04...",POINT (-77.03099 -12.04184),-12.041836,-77.030994
9406,15012301201039B,OU:DEPORTE,CAMPO DEPORTIVO,VISTA ALEGRE,NaN,PACHACAMAC,LIMA,LIMA,2117.49,187.80,...,039,B,150123,0001,1.683302e-07,0.001679,"POLYGON ((-76.88069 -12.09506, -76.88074 -12.0...",POINT (-76.88085 -12.09479),-12.094792,-76.880848
9407,150133029000240,OU:DEPORTE,COMPLEJO DEPORTIVO,IPD SAN JUAN DE MIRAFLORES,NaN,SAN JUAN DE MIRAFLORES,LIMA,LIMA,54665.69,980.13,...,024,0,150133,0001,1.176255e-05,0.014409,"POLYGON ((-76.97324 -12.1596, -76.9732 -12.158...",POINT (-76.97182 -12.16107),-12.161069,-76.971820


# Caracterización de Albergues Potenciales: Agregación de vulnerabilidad y riesgo sísmico

In [236]:
df_manzanas_carac = pd.read_excel('BD_ManzanasCarac.xlsx')

In [237]:
df_manzanas_carac.shape

(106991, 13)

In [238]:
df_manzanas_carac.head()

,ID_MANZANA,NIV_VULNE,DESC_VULNE,NOMBDIST_VULN,NIV_RIESGO,DESC_RIESG,TOT_POB17,TOT_POB20,DENS_POB17,DENS_POB20,COB_AP17,COB_EE17,TIPO_EE17
0,150101001000010,BAJA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,LIMA,MEDIO,RIESGO DEL MEDIO URBANO ANTE SISMO,164.0,170.0,0.065346,0.065346,CON COBERTURA,CON COBERTURA,1.0
1,150101001000020,SIN VULNE,SIN VULNERABILIDAD,LIMA,SIN RIESGO,SIN RIESGO,4.0,4.0,0.001985,0.001985,CON COBERTURA,CON COBERTURA,1.0
2,150101001000030,SIN VULNE,SIN VULNERABILIDAD,LIMA,SIN RIESGO,SIN RIESGO,56.0,58.0,0.005767,0.005767,CON COBERTURA,CON COBERTURA,1.0
3,150101001000040,MEDIA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,LIMA,MEDIO,RIESGO DEL MEDIO URBANO ANTE SISMO,235.0,244.0,0.043450,0.043450,CON COBERTURA,CON COBERTURA,1.0
4,150101001000050,ALTA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,LIMA,ALTO,RIESGO DEL MEDIO URBANO ANTE SISMO,361.0,374.0,0.071250,0.071250,CON COBERTURA,CON COBERTURA,1.0


In [239]:
# Cambiar el nombre de cobertura de Agua y Electricidad
df_manzanas_carac = df_manzanas_carac.rename(columns={'COB_AP17': 'COB_AGUA','COB_EE17': 'COB_ELEC'})

In [242]:
# Convertir la columna ID_MANZANA a string
df_manzanas_carac['ID_MANZANA'] = df_manzanas_carac['ID_MANZANA'].astype(str)

In [243]:
# CRUZAR DATOS DE SHAPEFILE
# Realizar el inner join entre df_manzanas_carac y gdf_combined_coor
# La clave de unión es "ID_MANZANA"
gdf_albergues_carac = df_manzanas_carac.merge(gdf_combined_coor, left_on='ID_MANZANA', right_on='ID_MANZANA', how='inner')

In [244]:
print('Albergues potenciales caracterizados: ', gdf_albergues_carac.shape[0])

Albergues potenciales caracterizados:  7420


In [246]:
gdf_albergues_carac.head(3)

,ID_MANZANA,NIV_VULNE,DESC_VULNE,NOMBDIST_VULN,NIV_RIESGO,DESC_RIESG,TOT_POB17,TOT_POB20,DENS_POB17,DENS_POB20,...,CODMZNA,SUFMZNA,UBIGEO,CODCCPP,Shape_STAr,Shape_STLe,geometry,centroid,latitud,longitud
0,150101001000020,SIN VULNE,SIN VULNERABILIDAD,LIMA,SIN RIESGO,SIN RIESGO,4.0,4.0,0.001985,0.001985,...,002,0,150101,0001,1.753006e-07,0.001879,"POLYGON ((-77.08217 -12.03634, -77.0821 -12.03...",POINT (-77.08182 -12.03641),-12.036411,-77.081824
1,150101001000060,BAJA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,LIMA,MEDIO,RIESGO DEL MEDIO URBANO ANTE SISMO,0.0,0.0,0.000000,0.000000,...,006,0,150101,0001,4.451045e-07,0.002856,"POLYGON ((-77.07984 -12.03645, -77.0798 -12.03...",POINT (-77.08031 -12.03674),-12.036744,-77.080308
2,150101001000200,MEDIA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,LIMA,MEDIO,RIESGO DEL MEDIO URBANO ANTE SISMO,132.0,137.0,0.016823,0.016823,...,020,0,150101,0001,6.615369e-07,0.003927,"POLYGON ((-77.07973 -12.03763, -77.07969 -12.0...",POINT (-77.08021 -12.03807),-12.038073,-77.080213


In [254]:
gdf_albergues_carac['GEOMETRY'] = gdf_albergues_carac.geometry
# Eliminar múltiples columnas que no guardan información relevante
gdf_albergues_carac = gdf_albergues_carac.drop(columns=['geometry'])

In [261]:
# Checar las columnas con nombre de mas de 10 caracteres
for col in list(gdf_albergues_carac.columns):
    if len(col)>10:
        print(col,len(col))

NOMBDIST_VULN 13


In [262]:
# La columna NOMBDIST_VULN no tiene importancia, solo tiene un dato: 'LIMA', se procede a ELIMINAR
gdf_albergues_carac = gdf_albergues_carac.drop(columns=['NOMBDIST_VULN'])

In [269]:
# Eliminar la columna 'centroid' ya que no puede haber 2 columnas de tipo geometry
gdf_albergues_carac = gdf_albergues_carac.drop(columns=['centroid'])

In [270]:
# Descargar el archivo de los albergues potenciales como SHAPEFILE
gdf_albergues_carac = gpd.GeoDataFrame(gdf_albergues_carac, geometry='GEOMETRY')

In [272]:
# Exportar a shapefile
gdf_albergues_carac.to_file('albergues_caracterizados_lima.shp', driver='ESRI Shapefile')

In [209]:
df_bd = pd.read_excel('BD_completa.xlsx')

In [210]:
df_bd

,ID_MANZANA,NIV_VULNE,DESC_VULNE,NOMBDIST_VULN,NIV_RIESGO,DESC_RIESG,TOT_POB17,TOT_POB20,DENS_POB17,DENS_POB20,INSTITUTO_ES,NIV_AT_ES,GEST_ES,CAT_ES,NCAT_ES,NOM_EQ,POB_ATEN_20,COB_AP17,COB_EE17,TIPO_EE17
0,15010300200031C,ALTA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,ATE,ALTO,RIESGO DEL MEDIO URBANO ANTE SISMO,0.0,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO DATA,NO DATA,3.0
1,15010302600036B,ALTA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,ATE,ALTO,RIESGO DEL MEDIO URBANO ANTE SISMO,28.0,29.0,0.013883,0.013883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CON COBERTURA,CON COBERTURA,1.0
2,150111001000010,SIN VULNE,SIN VULNERABILIDAD,EL AGUSTINO,SIN RIESGO,SIN RIESGO,0.0,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO DATA,NO DATA,3.0
3,150111001000030,ALTA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,EL AGUSTINO,ALTO,RIESGO DEL MEDIO URBANO ANTE SISMO,66.0,68.0,0.037133,0.037133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CON COBERTURA,CON COBERTURA,1.0
4,150111001000040,MUY ALTA,VULNERABILIDAD SINTESIS DEL MEDIO URBANO ANTE ...,EL AGUSTINO,ALTO,RIESGO DEL MEDIO URBANO ANTE SISMO,86.0,89.0,0.061791,0.061791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CON COBERTURA,CON COBERTURA,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112854,150101006000010,SIN VULNE,SIN VULNERABILIDAD,LIMA,SIN RIESGO,SIN RIESGO,67.0,69.0,0.000096,0.000096,OTRO,PRIMER NIVEL,PUBLICO,CENTRO DE SALUD SIN INTERNAMIENTO,I-3,SERVICIOS MÉDICOS UNMSM,10000.0,NO DATA,NO DATA,3.0
112855,150101006000010,SIN VULNE,SIN VULNERABILIDAD,LIMA,SIN RIESGO,SIN RIESGO,67.0,69.0,0.000096,0.000096,MINSA,PRIMER NIVEL,PUBLICO,CENTRO DE SALUD SIN INTERNAMIENTO,I-3,CENTRO DE SALUD MENTAL COMUNITARIO UNIVERSITAR...,10000.0,NO DATA,NO DATA,3.0
112856,150101006000010,SIN VULNE,SIN VULNERABILIDAD,LIMA,SIN RIESGO,SIN RIESGO,67.0,69.0,0.000096,0.000096,OTRO,PRIMER NIVEL,PUBLICO,CENTRO DE SALUD SIN INTERNAMIENTO,I-3,"INSTITUTO DE MEDICINA TROPICAL ""DANIEL ALCIDES...",10000.0,NO DATA,NO DATA,3.0
112857,150140012000110,SIN VULNE,SIN VULNERABILIDAD,SANTIAGO DE SURCO,SIN RIESGO,SIN RIESGO,0.0,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO DATA,NO DATA,3.0


In [192]:
# Convertir la columna ID_MANZANA a string
df_bd['ID_MANZANA'] = df_bd['ID_MANZANA'].astype(str)

In [201]:
# CRUZAR DATOS DE SHAPEFILE
# Realizar el inner join entre gdf_Lima y df_recreacion_filter
# La clave de unión es "ID_MANZANA"
gdf_carac = gdf_combined.merge(df_bd, left_on='ID_MANZANA', right_on='ID_MANZANA', how='inner')

In [195]:
gdf_combined.shape

(7420, 34)

In [211]:
# DATOS QUE NO SE CRUZAN
# Realizar el outer join entre gdf_Lima y df_recreacion_filter
gdf_carac_b = gdf_combined.merge(df_bd, left_on='ID_MANZANA', right_on='ID_MANZANA', how='outer', indicator=True)

# Filtrar los registros que no encontraron coincidencia
aaa = gdf_carac_b[gdf_carac_b['_merge'] == 'right_only']

In [212]:
aaa

,OBJECTID,COD_EQ,ID_MANZANA,IDLOTE,IDMZNAR,COD_SECT,TIPO_EQ,SUBTIPO_EQ,NOM_EQ_x,AREA_M2,...,NIV_AT_ES,GEST_ES,CAT_ES,NCAT_ES,NOM_EQ_y,POB_ATEN_20,COB_AP17,COB_EE17,TIPO_EE17,_merge
8029,NaN,NaN,15010300200031C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NO DATA,NO DATA,3.0,right_only
8030,NaN,NaN,15010302600036B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,CON COBERTURA,CON COBERTURA,1.0,right_only
8031,NaN,NaN,150111001000030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,CON COBERTURA,CON COBERTURA,1.0,right_only
8032,NaN,NaN,150111001000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,CON COBERTURA,CON COBERTURA,1.0,right_only
8033,NaN,NaN,150111001000050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,CON COBERTURA,CON COBERTURA,1.0,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112854,NaN,NaN,150131004000200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,CON COBERTURA,CON COBERTURA,1.0,right_only
112855,NaN,NaN,150101006000010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PRIMER NIVEL,PUBLICO,CENTRO DE SALUD SIN INTERNAMIENTO,I-3,SERVICIOS MÉDICOS UNMSM,10000.0,NO DATA,NO DATA,3.0,right_only
112856,NaN,NaN,150101006000010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PRIMER NIVEL,PUBLICO,CENTRO DE SALUD SIN INTERNAMIENTO,I-3,CENTRO DE SALUD MENTAL COMUNITARIO UNIVERSITAR...,10000.0,NO DATA,NO DATA,3.0,right_only
112857,NaN,NaN,150101006000010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,PRIMER NIVEL,PUBLICO,CENTRO DE SALUD SIN INTERNAMIENTO,I-3,"INSTITUTO DE MEDICINA TROPICAL ""DANIEL ALCIDES...",10000.0,NO DATA,NO DATA,3.0,right_only
